In [ ]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
import os 
from random import shuffle
from tqdm import tqdm
TRAIN_DIR = 'D:/try/train'
IMG_SIZE = 250


In [ ]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.imread(path)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append(img)
    return training_data

In [ ]:
train_data = create_train_data()

In [ ]:
import xlrd
book = xlrd.open_workbook('angles.xls')
sheet = book.sheet_by_name('Sheet1')
labels = [[sheet.cell_value(r, c) for c in range(1,7)] for r in range(1,sheet.nrows)]
# Profit !
print(len(labels)) 

In [ ]:
import numpy as np
def normalize(x):
    """
        argument
            - x: input image data in numpy array [32, 32, 3]
        return
            - normalized x 
    """
    min_val = np.min(x)
    max_val = np.max(x)
    x = (x-min_val) / (max_val-min_val)
    return x

In [ ]:
normalize(train_data)

In [ ]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import problem_unittests as tests
import helper
import numpy as np

In [ ]:
import tensorflow as tf

In [ ]:
# Remove previous weights, bias, inputs, etc..
import tensorflow as tf
tf.reset_default_graph()

# Inputs
x = tf.placeholder(tf.float32, shape=(250, 250, 3), name='input_x')
y =  tf.placeholder(tf.float32, shape=(6), name='output_y')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

In [ ]:
def conv2d_maxpool(x_tensor, conv_num_outputs, conv_ksize, conv_strides, pool_ksize, pool_strides):
  
    num_input_channels = x_tensor.shape.as_list()[3]
    num_filters = conv_num_outputs
    print("x_tensor shape: {} conv_num_outpus: {} conv_ksize: {}\
            conv_strides: {} pool_ksize: {} pool_strides: {}".format(x_tensor.shape, conv_num_outputs,
                                                                      conv_ksize, conv_strides, pool_ksize, pool_strides)) 
    
    shape = [conv_ksize[0], conv_ksize[1], num_input_channels, num_filters]
    print("shape: ",shape)
    weight = tf.Variable(tf.truncated_normal(shape, mean=0, stddev=0.08))
    bias = tf.Variable(tf.constant(0.05, shape=[num_filters]))
    #bias = tf.Variable(tf.zeros(num_filters))
    conv = tf.nn.conv2d(x_tensor, weight, strides=[1,conv_strides[0], conv_strides[1],1], padding='SAME')
    conv = tf.nn.bias_add(conv, bias)
    conv = tf.nn.relu(conv)
    return tf.nn.max_pool(conv, ksize=[1,pool_ksize[0],pool_ksize[1],1], 
                  strides=[1,pool_strides[0], pool_strides[1], 1],
                  padding='SAME')


tests.test_con_pool(conv2d_maxpool)

In [ ]:
def flatten(x_tensor):
   
    # TODO: Implement Function
    #print(x_tensor)
    shape = x_tensor.get_shape()
    num_features = shape[1:4].num_elements()
    layer_flat = tf.reshape(x_tensor, [-1, num_features])
    print("shape: {} num_feautres: {}".format(shape, layer_flat.get_shape().as_list()[1]))
    return layer_flat


tests.test_flatten(flatten)

In [ ]:
def fully_conn(x_tensor, num_outputs):
   
    # TODO: Implement Function
    print(x_tensor.get_shape())
    weights = tf.Variable(tf.truncated_normal([x_tensor.get_shape().as_list()[1], num_outputs], mean=0.0, stddev=0.08))
    bias = tf.Variable(tf.constant(0.05, shape=[num_outputs]))
    #bias = tf.Variable(tf.zeros(num_outputs))
    tf.Variable(tf.zeros(num_outputs))
    fc = tf.matmul(x_tensor, weights) + bias
    return tf.nn.relu(fc)



tests.test_fully_conn(fully_conn)

In [ ]:
def output(x_tensor, num_outputs):

    # TODO: Implement Function
    weights = tf.Variable(tf.truncated_normal([x_tensor.get_shape().as_list()[1], num_outputs],mean=0.0,stddev=0.5))
    bias = tf.Variable(tf.constant(0.05, shape=[num_outputs]))
    #bias = tf.Variable(tf.zeros(num_outputs))
    out = tf.add(tf.matmul(x_tensor, weights),bias)
    return out



tests.test_output(output)

In [ ]:
def conv_net(x, keep_prob):
    
    
    
   
    conv1 = conv2d1_maxpool(x, 128, [5,5], [2,2], [3,3], [2,2])
    conv1 = tf.nn.dropout(conv1, keep_prob)
    conv2 = conv2d1_maxpool(conv1, 256, [3,3], [1,1], [1,1], [2,2])
    #conv2 = tf.nn.dropout(conv2, keep_prob)
    #conv3 = conv2d_maxpool(conv2, 128, [5,5], [2,2], [3,3], [2,2])
    # TODO: Apply a Flatten Layer
    # Function Definition from Above:
    #   flatten(x_tensor)
    fl1=flatten(conv2)
    print("fl1: ",fl1," : ",fl1.get_shape().as_list()[1])
    # TODO: Apply 1, 2, or 3 Fully Connected Layers
    #    Play around with different number of outputs
    # Function Definition from Above:
    #   fully_conn(x_tensor, num_outputs)
    fc1 = fully_conn(fl1,500)
    fc1 = tf.nn.dropout(fc1, keep_prob)
    fc2 = fully_conn(fc1,100)
    #tf.nn.dropout(fc1, keep_prob)
    # TODO: Apply an Output Layer
    #    Set this to the number of classes
    # Function Definition from Above:
    #   output(x_tensor, num_outputs)
    out = output(fc2,6)
    return out




# Remove previous weights, bias, inputs, etc..
tf.reset_default_graph()

# Inputs
x = neural_net_image_input((3624, 4672, 3))
y = neural_net_label_input(6)
keep_prob = neural_net_keep_prob_input()

# Model
logits = conv_net(x, keep_prob)

# Name logits Tensor, so that is can be loaded from disk after training
logits = tf.identity(logits, name='logits')

# Loss and Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer().minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

test_conv_net(conv_net)

In [ ]:
def train_neural_network(session, optimizer, keep_probability, feature_batch, label_batch):
   
    # TODO: Implement Function
    session.run(optimizer, feed_dict={
        x: feature_batch,
        y : label_batch,
        keep_prob : keep_probability
    })



tests.test_train_nn(train_neural_network)

In [ ]:
def print_stats(session, feature_batch, label_batch, cost, accuracy):
   
    # TODO: Implement Function
    loss = session.run(cost, feed_dict={ 
        x : feature_batch,
        y : label_batch,
        keep_prob: 1.0
    })
    validation_accuracy = sess.run(accuracy, feed_dict={
         x : feature_batch,
        y : label_batch,
        keep_prob: 1.0
    }) 
    print('Loss: {:>10.4f} Accuracy: {:.6f}'.format(loss,validation_accuracy))

In [ ]:
epochs = 20
batch_size = 256
keep_probability = 0.5

In [ ]:
import xlrd
book = xlrd.open_workbook('angles.xls')
sheet = book.sheet_by_name('Sheet1')
labels = [[sheet.cell_value(r, c) for c in range(1,7)] for r in range(1,sheet.nrows)]
# Profit !
print(len(labels)) 
print('Checking the Training on a Single Batch...')
with tf.Session() as sess:
    # Initializing the variables
    sess.run(tf.global_variables_initializer())
    # Training cycle
    for epoch in range(epochs):
        batch_i = 1
        for train_data1, labels1 in zip(train_data, labels):
           # print (batch_features)
           # print ('>>> ', batch_labels)
            train_neural_network(sess, optimizer, keep_probability, train_data1,labels1)
        print('Epoch {:>2}, CIFAR-10 Batch {}:  '.format(epoch + 1, batch_i), end='')
        print_stats(sess, train_data1,labels1, cost, accuracy)